In [1]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd

In [2]:
# Importando os arquivos
jogos = pd.read_csv('../data/raw/matches_FLA2024.csv', parse_dates=['kick-off'])
gols = pd.read_csv('../data/raw/goal_scorers_2024.csv')

In [3]:
# Mostrando as 5 primeiras linhas do dataframe jogos
jogos.head()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211


In [4]:
# Mostrando as 5 ultimas linhas do dataframe jogos
jogos.tail()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance
37,BR13,Flamengo,Cruzeiro,2,1,Campeonato Brasileiro - Round 13,2024-06-30 18:30:00,Maracanã-RJ,52768
38,BR14,Atlético-MG,Flamengo,2,4,Campeonato Brasileiro - Round 14,2024-07-03 21:30:00,MRV Arena-MG,39993
39,BR15,Flamengo,Cuiabá,1,1,Campeonato Brasileiro - Round 15,2024-07-06 20:00:00,Maracanã-RJ,54948
40,BR16,Flamengo,Fortaleza,1,2,Campeonato Brasileiro - Round 16,2024-07-11 20:00:00,Maracanã-RJ,57500
41,BR17,Flamengo,Criciúma,2,1,Campeonato Brasileiro - Round 17,2024-07-20 16:00:00,Mané Garrincha-BR,60127


In [5]:
# Mostrando as 5 primeiras linhas do dataframe jogos
gols.head()

,id_match,gols,assist
0,CA1,Léo Pereira,Arrascaeta
1,CA1,Pedro,Gerson
2,CA1,Everton Cebolinha,NaN
3,CA1,Varela,Luiz Araújo
4,AM1,Pedro,NaN


In [6]:
# Mostrando as 5 ultimas linhas do dataframe jogos
gols.tail()

,id_match,gols,assist
70,BR14,Bruno Henrique,Pedro
71,BR15,Pedro,Ayrton Lucas
72,BR16,Pedro,NaN
73,BR17,Pedro,Arrascaeta
74,BR17,Gabi,NaN


### Criando a coluna torneio

In [7]:
# Split o nome do torneio e a rodada
lista = [i.split(' -', 1)[0] for i in jogos["tournament"].tolist()]

# Transformando em uma nova coluna chamada torneio
jogos['torneio'] = np.array(lista)

### Criando a coluna local (casa/fora)

In [8]:
def obter_locale(row):
    if row['torneio'] == 'Supercopa do Brasil' or row['torneio'] == 'Club World Championship':
        return 'Neutro'
    elif row['home_team'] == 'Flamengo':
        return 'Casa'
    else:
        return 'Fora'

    
# Aplicar a função para criar a nova coluna
jogos['locale'] = jogos.apply(obter_locale, axis=1)

### Criando a coluna resultado

In [9]:
# Função para obter resultado
def obter_resultado(row):
    if row['home_team'] == 'Flamengo':
        gols_marcados = row['home_score']
        gols_sofridos = row['away_score']
        
        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'
    if row['away_team'] == 'Flamengo':
        gols_marcados = row['away_score']
        gols_sofridos = row['home_score']

        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'

# Aplicar a função para criar a nova coluna
jogos['resultado'] = jogos.apply(obter_resultado, axis=1)

### Criando mais algumas colunas 

In [10]:
def gerar_estatisticas_gols(row):
    if row['home_team'] == 'Flamengo':
        gols_feitos_casa = row['home_score']
        gols_sofridos_casa = row['away_score']
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Sim' if row['home_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['away_score'] > 0 else 'Não'
    elif row['away_team'] == 'Flamengo':
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = row['away_score']
        gols_sofridos_fora = row['home_score']
        marcou_gol = 'Sim' if row['away_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['home_score'] > 0 else 'Não'
    else:
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Não'
        sofreu_gol = 'Não'

    return pd.Series([gols_feitos_casa, gols_sofridos_casa, gols_feitos_fora, gols_sofridos_fora, marcou_gol, sofreu_gol])

jogos[['gols_casa', 'gols_sofridos_casa', 'gols_fora', 'gols_sofridos_fora', 'marcou_gol', 'sofreu_gol']] = jogos.apply(gerar_estatisticas_gols, axis=1)        

jogos['jogos_cumulativos'] = range(1, len(jogos) + 1)

In [11]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,locale,resultado,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,Casa,Vitória,4,0,0,0,Sim,Não,1
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,Fora,Vitória,0,0,2,0,Sim,Não,2
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428,Carioca,Fora,Empate,0,0,1,1,Sim,Sim,3
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678,International Friendly Game,Fora,Empate,0,0,1,1,Sim,Sim,4
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211,Carioca,Fora,Empate,0,0,0,0,Não,Não,5
5,CA4,Sampaio Corrêa-RJ,Flamengo,0,2,Carioca - Round 4,2024-01-31 21:30:00,Mangueirão-PA,47644,Carioca,Fora,Vitória,0,0,2,0,Sim,Não,6
6,CA5,Vasco,Flamengo,0,0,Carioca - Round 5,2024-02-04 19:00:00,Maracanã-RJ,56318,Carioca,Fora,Empate,0,0,0,0,Não,Não,7
7,CA6,Flamengo,Botafogo,1,0,Carioca - Round 6,2024-02-07 21:30:00,Maracanã-RJ,46798,Carioca,Casa,Vitória,1,0,0,0,Sim,Não,8
8,CA7,Flamengo,Volta Redonda,3,0,Carioca - Round 7,2024-02-10 16:00:00,Maracanã-RJ,42373,Carioca,Casa,Vitória,3,0,0,0,Sim,Não,9
9,CA8,Bangu-RJ,Flamengo,0,3,Carioca - Round 8,2024-02-15 21:30:00,Arena Batistão-SE,15575,Carioca,Fora,Vitória,0,0,3,0,Sim,Não,10


#### Salvando os dataframes 

In [12]:
jogos.to_csv('../data/processed/matches_FLA2024.csv', index=False)
gols.to_csv('../data/processed/goal_scorers_2024.csv', index=False)